In [65]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.preprocessing import OneHotEncoder
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.utils import np_utils

In [2]:
# Import CIFAR100 small classification dataset

from keras.datasets import cifar100

(Xtrain, ytrain), (Xtest, ytest) = cifar100.load_data(label_mode = 'fine')

169009152/169001437 [==============================] - 110s 1us/step


In [6]:
# Defining early parameters

N, w, h, c = Xtrain.shape

In [13]:
# One hot encoding labels

encoder = OneHotEncoder()
ytrain_ind = encoder.fit_transform(ytrain).toarray()
ytest_ind = encoder.transform(ytest).toarray()

/Users/blagawuga/anaconda3/envs/py367/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:363: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [23]:
# Normalize input data (NOTE: - Original AlexNet paper didn't implement input Normalization, rather used layer normalization)

Xtrain = Xtrain/255.
Xtest = Xtest/255.

# AlexNet model

### __[Original AlexNet paper](https://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks)__

<img src="https://neurohive.io/wp-content/uploads/2018/10/AlexNet-1.png" alt="Model structure for AlexNet" title="Layer design of AlexNet" />

In [128]:
# Original AlexNet uses 224x224x3 RGB Images from ImageNet dataset, we'll be using 32x32x3 from CIFAR100 dataset

class AlexNet:
    """
        Sci-kit type based model for implementing AlexNet*
    """
    def __init__(self, n_classes, kernel_sizes, fc_layer_sizes, *input_shape):
        (self.N, self.w, self.h, self.c) = input_shape
        self.n_class = n_classes
        self.kernel_sizes = kernel_sizes
        self.fc_layer_sizes = fc_layer_sizes
        self.model = Sequential()
        assert len(kernel_sizes) == 5 and type(kernel_sizes) == list
        assert len(fc_layer_sizes) == 2 and type(fc_layer_sizes) == list
    
    def initialize_model(self):
        self.model.add(Convolution2D(filters=96, input_shape=(self.w, self.h, self.c), kernel_size=self.kernel_sizes[0], strides=(4,4), activation='relu', padding='valid', bias_initializer='ones'))
        self.model.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1)))
        self.model.add(ZeroPadding2D(padding=(2, 2), data_format='channels_last'))
        self.model.add(Convolution2D(filters=256, kernel_size=self.kernel_sizes[1], strides=(1,1), activation='relu', padding='valid', bias_initializer='ones'))
        self.model.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1)))
        self.model.add(ZeroPadding2D(padding=(1, 1), data_format='channels_last'))
        self.model.add(Convolution2D(filters=384, kernel_size=self.kernel_sizes[2], strides=(1,1), activation='relu', padding='valid', bias_initializer='ones'))
        self.model.add(ZeroPadding2D(padding=(1, 1), data_format='channels_last'))
        self.model.add(Convolution2D(filters=384, kernel_size=self.kernel_sizes[3], strides=(1,1), activation='relu', padding='valid', bias_initializer='ones'))
        self.model.add(ZeroPadding2D(padding=(1, 1), data_format='channels_last'))
        self.model.add(Convolution2D(filters=256, kernel_size=self.kernel_sizes[3], strides=(1,1), activation='relu', padding='valid', bias_initializer='ones'))
        self.model.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1)))
        self.model.add(Flatten())
        self.model.add(Dense(self.fc_layer_sizes[0]))
        self.model.add(Dropout(rate=0.5))
        self.model.add(Dense(self.fc_layer_sizes[1]))
        self.model.add(Dropout(rate=0.5))
        self.model.add(Dense(self.n_class, activation='softmax'))
        self.model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
    
    def train_model(self, X, y, batch_size, n_epochs, verbose=1):
        self.model.fit(X, y, batch_size=batch_size, epochs=n_epochs, verbose=verbose)
        
    def eval_model(self, Xtest, ytest, verbose):
        score = self.model.evaluate(Xtest, ytest, verbose=verbose)
        return score

In [ ]:
# Check score with CIFAR dataset

alexNet = AlexNet(100, [(11, 11), (5, 5), (3, 3), (3, 3), (3, 3)], [4096, 4096], *Xtrain.shape)
alexNet.initialize_model()
alexNet.train_model(Xtrain, ytrain_ind, 128, 10, 1)
score = alexNet.eval_model(Xtest, ytest_ind, verbose)